In [31]:
from Function.Function import *
from os.path import abspath , join
from sklearn.preprocessing import normalize ,LabelEncoder

from sklearn.decomposition import PCA 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
current_folder =abspath('');current_folder



'/Users/yangtao/Documents/GitHub/850-project'

# Preprocessing data

In [2]:
N_calibrate = 0.8 #  days for train
N_validation = 1 - N_calibrate # days for test
data = pd.read_csv('Data/finalproject_training.csv')

# the xs that was cleaned for useless price data and non number data and NONE data(first check)
data = data.loc[:,data.columns[12:]]
data = data.loc[:,(np.array(data.dtypes == 'int64') ) | \
                               (np.array(data.dtypes == 'float64') )].dropna()

In [3]:
data

,Bullish,Neutral,Bearish,Bullish8WeekMovAvg,SP500WeeklyHigh,SP500WeeklyLow,SP500WeeklyClose,fyearq,fqtr,actq,...,oancfy,txpdy,dvpspq,dvpsxq,mkvaltq,ggroup,gind,gsector,gsubind,sic
0,0.294243,0.409382,0.296375,0.325835,2193.42,2168.50,2175.44,2016,3,3530.000,...,559.000,54.000,0.1150,0.1150,15587.6400,3520,352030,35,35203010,3826
8,0.294243,0.409382,0.296375,0.325835,2193.42,2168.50,2175.44,2016,3,93761.000,...,49698.000,8990.000,0.5700,0.5700,515586.5740,4520,452020,45,45202030,3663
34,0.294243,0.409382,0.296375,0.325835,2193.42,2168.50,2175.44,2016,2,263.164,...,-2.853,2.578,0.0000,0.0000,1175.5510,3520,352010,35,35201010,2836
127,0.294243,0.409382,0.296375,0.325835,2193.42,2168.50,2175.44,2016,3,7399.000,...,1669.000,40.000,0.1000,0.1000,28416.7033,4530,453010,45,45301010,3559
259,0.294243,0.409382,0.296375,0.325835,2193.42,2168.50,2175.44,2016,4,140.414,...,-0.126,1.964,0.0000,0.0000,40.9306,4520,452010,45,45201020,3663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105196,0.261307,0.316583,0.422111,0.296648,2939.08,2834.97,2887.94,2019,2,1295.382,...,352.175,62.214,0.0000,0.0000,24795.4997,4510,451020,45,45102030,7370
105226,0.261307,0.316583,0.422111,0.296648,2939.08,2834.97,2887.94,2019,4,8477.000,...,1547.000,377.000,0.5000,0.5000,14027.2500,4520,452020,45,45202030,3572
105283,0.261307,0.316583,0.422111,0.296648,2939.08,2834.97,2887.94,2019,3,1298.109,...,219.202,54.757,0.1625,0.1625,6973.8245,2010,201060,20,20106020,3620
105298,0.261307,0.316583,0.422111,0.296648,2939.08,2834.97,2887.94,2019,1,3589.568,...,298.216,1.661,0.3700,0.3700,29600.2784,4530,453010,45,45301020,3674


In [7]:
#normalization or standardlization(i did normalization here)
data = pd.DataFrame(normalize(data, axis=0) , columns = data.columns , index = data.index)
# calibrate and standlization
X_data ,Y_data =get_data(data,N_calibrate , N_validation)

In [8]:
X_data['calibrate']['processed'].columns

Index(['Bullish', 'Neutral', 'Bearish', 'Bullish8WeekMovAvg',
       'SP500WeeklyHigh', 'SP500WeeklyLow', 'SP500WeeklyClose', 'fyearq',
       'fqtr', 'actq', 'ceqq', 'cheq', 'chq', 'ciq', 'csh12q', 'cshfd12',
       'cshfdq', 'cshopq', 'cshprq', 'dd1q', 'dlttq', 'dpq', 'epsf12',
       'epsfxq', 'epspxq', 'epsx12', 'esopctq', 'gdwlq', 'ibadjq', 'ibcomq',
       'ibmiiq', 'ibq', 'intanq', 'invtq', 'lctq', 'lltq', 'loq', 'lseq',
       'ltq', 'niq', 'nopiq', 'oiadpq', 'oibdpq', 'piq', 'rdipq', 'rectq',
       'req', 'revtq', 'tfvaq', 'tfvceq', 'tfvlq', 'txdbclq', 'txtq', 'xaccq',
       'xintq', 'xoprq', 'xrdq', 'fincfy', 'intpny', 'ivncfy', 'oancfy',
       'txpdy', 'dvpspq', 'dvpsxq', 'mkvaltq', 'ggroup', 'gind', 'gsector',
       'gsubind', 'sic'],
      dtype='object')

# Autoencoder or PCA ?

# lets do Autoencoder with nerual network first

# Construct model

In [10]:
dimension_of_input  = len(X_data['calibrate']['processed'].T) # number of stock
autoencoder = inital_model(dimension_of_input)
# train model
# we use net difference to train model
data = np.array(X_data['calibrate']['processed']) 
autoencoder.fit(data, data, shuffle=False, epochs=500, batch_size = 10 , verbose = 0)

In [11]:
autoencoder.save(join(current_folder,'Models/com_autoencoder_{}.h5'.format(1)))

# Predict and see what the factors is most relevent and explain most variance
# dimension reduction


In [13]:
path = join(current_folder,'Models/com_autoencoder_{}.h5'.format(1))
autoencoder = load_model(path)
predict = autoencoder.predict(data) 
# use different error measure to decide which factors explain most variance

# we need to figure out what is the best error measure,?

# sum absolute relative error
communal_information = np.sum(np.absolute((data - predict)),axis = 0) 
# ranking
ranking = np.array(communal_information).argsort()
ranks = pd.DataFrame({'communal' : communal_information[ranking] ,'index' : X_data['calibrate']['processed'].columns[ranking] })
ranks

,communal,index
0,0.018323,fyearq
1,1.927162,txdbclq
2,2.920242,SP500WeeklyLow
3,2.961752,SP500WeeklyClose
4,3.230478,SP500WeeklyHigh
...,...,...
65,19.160861,gdwlq
66,20.581846,xintq
67,22.069884,intpny
68,24.697313,epsf12


### lower the communal_information,higer explaination

### you can do prediction by using autoencoder directly. i can write it down if we decide to do it

# Lets do PCA
### from here you either use pca classifier or linear regression for prediction


In [67]:
#pca classifier
X = data
y = np.array(Y_data['calibrate']['processed'])

pca = PCA(2) #initial model

X_reduced = pca.fit_transform(X) # pca decomposition


rf = RandomForestClassifier() # initial predicition model
lab_enc = LabelEncoder()
y = lab_enc.fit_transform(y) # transfer data

X_train, X_test, y_train, y_test = train_test_split(X_reduced, y) # splite test and train


rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [68]:
# the score looks good to me
rf.score(X_test,y_test)

0.8681898066783831

# using pca in LinearRegression

In [69]:
from sklearn.linear_model import LinearRegression
X = data
y = np.array(Y_data['calibrate']['processed'])
pca = PCA(n_components=8) #initial model
X_pca = pca.fit_transform(X) # pca decomposition

model = LinearRegression()# initial predicition model
model.fit(X_pca,y) # fit

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [73]:
X_test =  np.array(X_data['validation']['processed']) 
X_test_pca = pca.fit_transform(X)
y_test = np.array(Y_data['validation']['processed'])

In [74]:
# out of sample, the score looks good to me
model.score(X_test_pca,y_test)

0.946570994225944

## save space for autoencoder